In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopy.distance

# 파일 경로와 범위 설정
base_path_2024 = "/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/"
file_template_2024 = "2024-10-05_{:02d}_인구종합.csv"
output_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/지도결과/"

# 출력 폴더가 없으면 생성
os.makedirs(output_path, exist_ok=True)

# 기준 좌표 (위도, 경도) 설정
center_lat = 37.5234376670226  # 중심 지역 좌표
center_lon = 126.93214353454457
quiet_area_lat = 37.519590  # 한산한 지역 좌표
quiet_area_lon = 126.926318

# 시간대별로 인구수를 비교하는 그래프를 생성하는 함수
def generate_population_comparison_graph(file_template, date_label):
    time_population_center_area = []  # 중심 지역의 시간별 인구수 저장
    time_population_quiet_area = []  # 한산한 지역의 시간별 인구수 저장

    # 00시부터 23시까지 처리
    for hour in range(24):
        file_path = base_path_2024 + file_template.format(hour)  # 파일 경로 생성
        try:
            # 파일 읽기
            df = pd.read_csv(file_path)

            # 각 지역에 해당하는 인구 수 필터링
            filtered_data_center = []  # 중심 지역 인구수 저장
            filtered_data_quiet = []  # 한산한 지역 인구수 저장

            # 각 행의 좌표와 인구수를 읽어오기
            for _, row in df.iterrows():
                x, y, popl = row['X_COORD'], row['Y_COORD'], row['POPL_CNT']

                if not pd.isna(x) and not pd.isna(y):
                    # 중심 지역과의 거리 계산 (200m 이내)
                    dist_center = geopy.distance.distance((center_lat, center_lon), (y, x)).km
                    if dist_center <= 0.2:  # 200미터 이내
                        filtered_data_center.append(popl)

                    # 한산한 지역과의 거리 계산 (200m 이내)
                    dist_quiet = geopy.distance.distance((quiet_area_lat, quiet_area_lon), (y, x)).km
                    if dist_quiet <= 0.2:  # 200미터 이내
                        filtered_data_quiet.append(popl)

            # 중심 지역과 한산한 지역의 총 인구수 계산
            total_population_center = sum(filtered_data_center)
            total_population_quiet = sum(filtered_data_quiet)

            # 결과 저장
            time_population_center_area.append((hour, total_population_center))
            time_population_quiet_area.append((hour, total_population_quiet))

        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            continue

    # 시간별 인구수 데이터로 DataFrame 생성
    time_population_df_center_area = pd.DataFrame(time_population_center_area, columns=['Hour', 'Total Population Center Area'])
    time_population_df_quiet_area = pd.DataFrame(time_population_quiet_area, columns=['Hour', 'Total Population Quiet Area'])

    # 단일 축에서 비교 그래프 그리기
    plt.figure(figsize=(10, 6))

    # 중심 지역과 조용한 지역에 대한 그래프 그리기
    plt.plot(time_population_df_center_area['Hour'], time_population_df_center_area['Total Population Center Area'],
             marker='x', color='b', linestyle='--', linewidth=2, markersize=6, label="Center Area")
    plt.plot(time_population_df_quiet_area['Hour'], time_population_df_quiet_area['Total Population Quiet Area'],
             marker='s', color='g', linestyle='-.', linewidth=2, markersize=6, label="Quiet Area")

    # 제목과 레이블 설정
    plt.title(f'Population Comparison by Hour ({date_label})', fontsize=14)
    plt.xlabel('Hour of the Day', fontsize=12)
    plt.ylabel('Total Population', fontsize=12)
    plt.xticks(range(24))
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.tight_layout()

    # 비교 그래프 이미지로 저장
    graph_file_path = output_path + f"population_comparison_center_vs_quiet_single_axis_2024.png"
    plt.savefig(graph_file_path)

    # 그래프 표시
    plt.show()

    # 그래프 파일 경로 출력
    print(f"그래프가 저장되었습니다: {graph_file_path}")

# 2024년 10월 5일의 데이터에 대해 비교 그래프 생성
generate_population_comparison_graph(file_template_2024, "2024-10-05")